In [ ]:
import json
import os
import ast
import csv
import io
from io import StringIO, BytesIO, TextIOWrapper
import gzip
from datetime import datetime, date
from s3_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import ast
from datetime import timedelta
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import warnings
import sys
import time
import torch
from statsmodels.tsa.stattools import adfuller, acf
from utils import *
from src.stationary_test_utils import *

In [ ]:

kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')

feature_name = kalman_df.columns.to_list()
feature_name = ['answer_text', 'hr_average', 'rmssd']
feature_name = list(filter(lambda item: "user" not in item.lower() and "date" not in item.lower() and "X" != item, feature_name))
print(feature_name)

# window_select(kalman_df, feature_name, ADFTestResultRetriver(kalman_df, feature_name))
# window_select(kalman_df, feature_name, ADFSingleWindowSelector(kalman_df, feature_name))
window_select(kalman_df, feature_name, ACFSingleWindowSelector(kalman_df, feature_name, method='avg'), minimal_window_size=5)


In [ ]:

# def window_select_acf(df, features):
#     acfs = []
#     for feature in features:
#         series = df[feature]
#         acfs.append(torch.abs(torch.Tensor(acf(series, nlags=len(series) - 1))))
#     print(acfs)
# window_select_acf(kalman_df, feature_name)
def get_acf_list(df, features, start_window, curr_window_size):
    acf_value_list = []
    for feature in features:
        series = df[start_window:][feature]
        acf_values = torch.abs(torch.Tensor(acf(series, nlags=len(series) - 1)))
        print(acf_values)
        
        acf_value_list.append(acf_values[:curr_window_size])
    acf_value_list = torch.stack(acf_value_list)
    print(acf_value_list.shape)
    print(torch.min(acf_value_list, axis=0)[0])
    return acf_value_list
# def window_select(df, features, minimal_window_size=7, get_value_list=get_acf_list, constraint_test=):
#     curr_window_size = minimal_window_size
#     total_len = len(df)
#     start_window = 0
#     p_value_list = []
#     break_points = []
#     while start_window + curr_window_size < total_len:
#         p_value_list = get_value_list(df, features, start_window, curr_window_size)
#         constraint_satisfied = constraint_test(p_value_list, "avg")
#         if not constraint_satisfied:
#             break_points.append(start_window)
#             start_window = start_window + curr_window_size
#             curr_window_size = minimal_window_size
#         else:
#             curr_window_size += 1
#     return break_points
get_acf_list(kalman_df, ['hr_average', 'rmssd'], 0, 8)

In [ ]:
get_acf_list(kalman_df, ['hr_average'], 14, len(kalman_df)-14)

In [ ]:
def p_value_constraint_test(p_value_list, method="max"):
    if method == "max":
        max_p_value = max(p_value_list)
        print(p_value_list)
        return max_p_value <= 0.05
    if method == "avg":
        print(np.mean(p_value_list))
        return np.mean(p_value_list) <= 0.05

def get_p_value_list(df, features, start_window, curr_window_size)
    p_value_list = []
    for feature in features:
        series = df[start_window:start_window+curr_window_size][feature]
        p_value = adfuller(series, autolag='AIC')[1]
        p_value_list.append(p_value)
    return p_value_list
    
def window_select_adf(df, features):
    minimal_window_size = 7
    curr_window_size = minimal_window_size
    total_len = len(df)
    start_window = 0
    p_value_list = []
    break_points = []
    while start_window + curr_window_size < total_len:
        p_value_list = get_p_value_list(df, features, start_window, curr_window_size)
        constraint_satisfied = p_value_constraint_test(p_value_list, "avg")
        if not constraint_satisfied:
            break_points.append(start_window)
            start_window = start_window + curr_window_size
            curr_window_size = minimal_window_size
        else:
            curr_window_size += 1
    return break_points
window_select_adf(kalman_df, feature_name)

In [ ]:
mean_df = pd.read_csv('./imputed_df_sleep_fatigue/mean_impute_df_userid_1032.csv')

In [ ]:
def stationary_test(series):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'n_lags: {result[1]}')
    print(f'p-value: {result[1]}')
    for key, value in result[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')


stationary_test(kalman_df[0:8]['answer_text'])


In [ ]:
stationary_test(kalman_df['hr_average'])


In [ ]:
stationary_test(kalman_df['breath_average'])

In [ ]:
stationary_test(kalman_df['rmssd'])


In [ ]:
stationary_test(mean_df['answer_text'])
stationary_test(mean_df['rmssd'])
stationary_test(mean_df['breath_average'])
stationary_test(mean_df['hr_average'])

In [ ]:
test_df = pd.read_csv('/domino/datasets/local/Bump/mice_user_42.csv')

In [ ]:
test_df

In [ ]:
import torch
from torch.utils import data
import matplotlib.pyplot as plt
import argparse
import math
import seaborn as sns; sns.set()
import sys

import numpy as np
import pickle
import os
import random

from statsmodels.tsa.stattools import adfuller
class TNCDataset(data.Dataset):
    def __init__(self, x, mc_sample_size, window_size, augmentation, epsilon=3, state=None, adf=False):
        super(TNCDataset, self).__init__()
        self.time_series = x
        self.T = x.shape[-1]
        self.window_size = window_size
        self.sliding_gap = int(window_size*25.2)
        self.window_per_sample = (self.T-2*self.window_size)//self.sliding_gap
        self.mc_sample_size = mc_sample_size
        self.state = state
        self.augmentation = augmentation
        self.adf = adf
        if not self.adf:
            self.epsilon = epsilon
            self.delta = 5*window_size*epsilon

    def __len__(self):
        return len(self.time_series)*self.augmentation

    def __getitem__(self, ind):
        ind = ind%len(self.time_series)
        print(2*self.window_size)
        print(self.T-2*self.window_size)
        t = np.random.randint(2*self.window_size, self.T-2*self.window_size)
        
        x_t = self.time_series[ind][:,t-self.window_size//2:t+self.window_size//2]
        plt.savefig('./plots/%s_seasonal.png'%ind)
        X_close = self._find_neighours(self.time_series[ind], t)
        X_distant = self._find_non_neighours(self.time_series[ind], t)

        if self.state is None:
            y_t = -1
        else:
            y_t = torch.round(torch.mean(self.state[ind][t-self.window_size//2:t+self.window_size//2]))
        return x_t, X_close, X_distant, y_t

    def _find_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if self.adf:
            gap = self.window_size
            corr = []
            for w_t in range(self.window_size,4*self.window_size, gap):
                try:
                    p_val = 0
                    for f in range(x.shape[-2]):
                        p = adfuller(np.array(x[f, max(0,t - w_t):min(x.shape[-1], t + w_t)].reshape(-1, )))[1]
                        p_val += 0.01 if math.isnan(p) else p
                    corr.append(p_val/x.shape[-2])
                except:
                    corr.append(0.6)
            self.epsilon = len(corr) if len(np.where(np.array(corr) >= 0.01)[0])==0 else (np.where(np.array(corr) >= 0.01)[0][0] + 1)
            self.delta = 5*self.epsilon*self.window_size

        ## Random from a Gaussian
        t_p = [int(t+np.random.randn()*self.epsilon*self.window_size) for _ in range(self.mc_sample_size)]
        t_p = [max(self.window_size//2+1,min(t_pp,T-self.window_size//2)) for t_pp in t_p]
        x_p = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_p])
        return x_p

    def _find_non_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if t>T/2:
            t_n = np.random.randint(self.window_size//2, max((t - self.delta + 1), self.window_size//2+1), self.mc_sample_size)
        else:
            t_n = np.random.randint(min((t + self.delta), (T - self.window_size-1)), (T - self.window_size//2), self.mc_sample_size)
        x_n = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_n])

        if len(x_n)==0:
            rand_t = np.random.randint(0,self.window_size//5)
            if t > T / 2:
                x_n = x[:,rand_t:rand_t+self.window_size].unsqueeze(0)
            else:
                x_n = x[:, T - rand_t - self.window_size:T - rand_t].unsqueeze(0)
        return x_n

In [ ]:
import pandas as pd
kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')
kalman_df


In [ ]:
ds = kalman_df[['hr_average', 'rmssd']]
ds = ds.to_numpy()
ds = ds.T

In [ ]:
mydataset = TNCDataset(ds, mc_sample_size=20, window_size=7, augmentation=1, adf=True)
mydataset[0]